# Lesson 4: Prompt Chaining for Financial Workflows - Demo

## Multi-Stage AI Workflows with Validation Gates

This demonstration shows how to build sophisticated financial AI workflows by chaining multiple specialized prompts together with Pydantic-based validation gates. We'll use a loan application processing workflow to show systematic, multi-stage analysis.

**Learning Objectives:**
- Understand prompt chaining methodology for complex workflows
- Learn to implement validation gates using Pydantic models
- Master sequential AI reasoning with quality control
- Observe how structured workflows improve reliability and auditability
- Practice building production-ready financial AI systems

## What We'll Demonstrate:

1. **Single Prompt Limitation**: Why complex tasks need multiple specialized prompts
2. **Three-Stage Chain Design**: Document Review → Risk Assessment → Decision Recommendation
3. **Pydantic Validation Gates**: Ensuring data quality between stages
4. **Error Handling**: Managing failures and retries in workflow chains
5. **Complete Workflow**: End-to-end loan processing automation
6. **Quality Comparison**: Chained vs. single-prompt approaches

In [2]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
from typing import List, Dict, Optional
from pydantic import BaseModel, Field, field_validator
from enum import Enum
import re

# Load environment variables from the root .env file
load_dotenv('../../.env')

True

In [3]:
# Setup OpenAI client for Vocareum environment
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

def get_completion(system_prompt, user_prompt, model="gpt-4o-mini"):
    """
    Function to get a completion from the OpenAI API.
    Args:
        system_prompt: The system prompt defining the AI's role
        user_prompt: The user's input or scenario  
        model: The model to use (default is gpt-4o-mini)
    Returns:
        completion text
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.3,  # Lower temperature for consistent structured output
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {e}"

## Loan Application Scenario

For this demonstration, we'll use a small business loan application that requires multi-stage analysis.

In [4]:
# Loan application scenario for workflow demonstration
loan_application = """
SMALL BUSINESS LOAN APPLICATION

Applicant Information:
- Business Name: Marina's Coastal Café
- Owner: Marina Santos, 38 years old
- Business Type: Restaurant/Café with seasonal tourism focus
- Location: Coastal town in Maine
- Years in Business: 4 years
- Legal Structure: LLC

Financial Information:
- Requested Loan Amount: $85,000
- Purpose: Equipment upgrade and seasonal inventory expansion
- Annual Revenue (Last 3 years): $180K, $220K, $195K
- Monthly Operating Expenses: $12,000 average
- Current Debt: $25,000 equipment loan (18 months remaining)
- Personal Credit Score: 685
- Business Checking Account: 4 years, average balance $8,500

Supporting Documents:
- 3 years tax returns (provided)
- Bank statements (6 months, provided)
- Business license (current, provided)
- Equipment quotes ($65,000 new kitchen equipment)
- Lease agreement (3 years remaining on current location)

Special Circumstances:
- Seasonal business (busy May-September, slower October-April)
- Recent decline in 2023 revenue due to local construction project affecting tourism
- Strong local reputation and customer loyalty
- Owner has 15 years restaurant industry experience
"""

print("📋 Loan application scenario loaded for workflow demonstration")
print("Focus: Multi-stage analysis with validation gates")

📋 Loan application scenario loaded for workflow demonstration
Focus: Multi-stage analysis with validation gates


## Problem: Single Prompt Limitation

First, let's see what happens when we try to handle this complex analysis with a single prompt.

In [5]:
# Single prompt approach - trying to do everything at once
single_prompt = """You are a loan underwriter. Analyze this loan application and provide:
1. Document completeness assessment
2. Financial risk analysis  
3. Final recommendation with loan terms"""

print("=== SINGLE PROMPT APPROACH ===")
single_response = get_completion(single_prompt, loan_application)
print(single_response)
print("\n" + "="*80)
print("❌ PROBLEMS WITH SINGLE PROMPT:")
print("- Lacks systematic analysis structure")
print("- No validation of intermediate steps") 
print("- Difficult to debug if something goes wrong")
print("- Inconsistent output format")
print("- Hard to modify or improve specific analysis areas")

=== SINGLE PROMPT APPROACH ===
### 1. Document Completeness Assessment

The loan application appears to be largely complete, with the following documents provided:

- **Tax Returns:** 3 years of tax returns have been submitted, which is essential for assessing the business's financial history and performance.
- **Bank Statements:** 6 months of bank statements are provided, allowing for an evaluation of cash flow and liquidity.
- **Business License:** A current business license is included, confirming the legitimacy of the business operation.
- **Equipment Quotes:** Quotes for the new kitchen equipment totaling $65,000 are provided, which aligns with the purpose of the loan.
- **Lease Agreement:** A lease agreement with 3 years remaining is included, ensuring the business has a stable location.

**Missing Documents:**
- **Personal Financial Statement:** A personal financial statement from the owner would provide insight into personal assets and liabilities.
- **Business Plan:** A detail

## Solution: Pydantic Models for Validation Gates

Let's define structured data models that ensure quality and consistency between workflow stages.

In [6]:
# Pydantic models for validation gates between stages

class DocumentStatus(str, Enum):
    COMPLETE = "complete"
    INCOMPLETE = "incomplete" 
    MISSING_CRITICAL = "missing_critical"

class RiskLevel(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"

class LoanDecision(str, Enum):
    APPROVE = "approve"
    CONDITIONAL_APPROVE = "conditional_approve"
    DECLINE = "decline"

# Stage 1 Output: Document Review
class DocumentReview(BaseModel):
    applicant_name: str = Field(..., description="Business owner name")
    business_name: str = Field(..., description="Legal business name")
    loan_amount: float = Field(..., gt=0, description="Requested loan amount")
    document_status: DocumentStatus = Field(..., description="Overall document completeness")
    missing_documents: List[str] = Field(default=[], description="List of missing required documents")
    financial_summary: Dict[str, float] = Field(..., description="Key financial metrics extracted")
    
    @field_validator('loan_amount')
    @classmethod
    def validate_loan_amount(cls, v):
        if v > 500000:
            raise ValueError('Loan amount exceeds maximum threshold')
        return v

# Stage 2 Output: Risk Assessment  
class RiskAssessment(BaseModel):
    overall_risk_level: RiskLevel = Field(..., description="Overall risk classification")
    credit_score: int = Field(..., ge=300, le=850, description="Personal credit score")
    debt_service_coverage: float = Field(..., description="Ability to service debt")
    risk_factors: List[str] = Field(..., description="Identified risk factors")
    mitigating_factors: List[str] = Field(..., description="Risk mitigation elements")
    risk_score: int = Field(..., ge=0, le=100, description="Quantitative risk score")
    
# Stage 3 Output: Final Decision
class LoanDecisionModel(BaseModel):
    decision: LoanDecision = Field(..., description="Final loan decision")
    approved_amount: Optional[float] = Field(None, description="Approved loan amount if approved")
    interest_rate: Optional[float] = Field(None, description="Proposed interest rate")
    term_months: Optional[int] = Field(None, description="Loan term in months")
    conditions: List[str] = Field(default=[], description="Special conditions or requirements")
    reasoning: str = Field(..., description="Detailed reasoning for decision")

print("✅ Pydantic validation models defined for workflow gates")

✅ Pydantic validation models defined for workflow gates


## Stage 1: Document Review and Data Extraction

First stage focuses exclusively on document completeness and data extraction.

In [7]:
# Stage 1: Document Review Specialist
stage1_prompt = """You are a loan documentation specialist. Your job is to review loan applications for document completeness and extract key financial data.

TASK: Analyze the provided loan application and extract structured information.

FOCUS ONLY ON:
1. Document completeness assessment
2. Basic data extraction
3. Identification of missing information

DO NOT provide risk analysis or loan recommendations - that's handled by other specialists.

Return ONLY a JSON object with this exact structure:
{
  "applicant_name": "string",
  "business_name": "string", 
  "loan_amount": number,
  "document_status": "complete|incomplete|missing_critical",
  "missing_documents": ["list", "of", "missing", "docs"],
  "financial_summary": {
    "annual_revenue_latest": number,
    "monthly_expenses": number,
    "current_debt": number,
    "credit_score": number
  }
}"""

def execute_stage1(application_text):
    """Execute Stage 1: Document Review with Gate Check"""
    print("\n=== STAGE 1: DOCUMENT REVIEW ===")
    
    response = get_completion(stage1_prompt, application_text)
    print(f"Raw AI Response:\n{response}\n")
    
    try:
        # Extract JSON from response
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            json_data = json.loads(json_match.group())
            
            # Validate with Pydantic
            document_review = DocumentReview(**json_data)
            print("✅ Stage 1 Gate Check: PASSED")
            return document_review
        else:
            raise ValueError("No valid JSON found in response")
            
    except (json.JSONDecodeError, Exception) as e:
        print(f"❌ Stage 1 Gate Check: FAILED - {e}")
        return None

# Test Stage 1
stage1_result = execute_stage1(loan_application)
if stage1_result:
    print(f"\n📊 Stage 1 Output Summary:")
    print(f"   Business: {stage1_result.business_name}")
    print(f"   Loan Amount: ${stage1_result.loan_amount:,.2f}")
    print(f"   Document Status: {stage1_result.document_status.value}")
    print(f"   Missing Docs: {len(stage1_result.missing_documents)} items")


=== STAGE 1: DOCUMENT REVIEW ===
Raw AI Response:
{
  "applicant_name": "Marina Santos",
  "business_name": "Marina's Coastal Café", 
  "loan_amount": 85000,
  "document_status": "complete",
  "missing_documents": [],
  "financial_summary": {
    "annual_revenue_latest": 195000,
    "monthly_expenses": 12000,
    "current_debt": 25000,
    "credit_score": 685
  }
}

✅ Stage 1 Gate Check: PASSED

📊 Stage 1 Output Summary:
   Business: Marina's Coastal Café
   Loan Amount: $85,000.00
   Document Status: complete
   Missing Docs: 0 items


## Stage 2: Risk Assessment Analysis

Second stage takes validated document data and performs specialized risk analysis.

In [8]:
# Stage 2: Risk Assessment Specialist
stage2_prompt = """You are a credit risk analyst specializing in small business loans. Analyze the provided business data for lending risk.

EVALUATION CRITERIA:
- Credit Score: 740+ (excellent), 680-739 (good), 620-679 (fair), <620 (poor)
- Debt Service Coverage: >1.25 (strong), 1.0-1.25 (acceptable), <1.0 (weak)
- Business Stability: Years in operation, revenue trends, industry factors
- Collateral and Personal Guarantees: Asset backing and owner commitment

RISK LEVELS:
- LOW: Strong financials, established business, excellent credit
- MEDIUM: Adequate financials with some concerns, manageable risks
- HIGH: Weak financials, significant risk factors, poor credit

Return ONLY a JSON object with this exact structure:
{
  "overall_risk_level": "low|medium|high",
  "credit_score": number,
  "debt_service_coverage": number, 
  "risk_factors": ["list", "of", "risks"],
  "mitigating_factors": ["list", "of", "positives"],
  "risk_score": number_0_to_100
}"""

def execute_stage2(document_review: DocumentReview):
    """Execute Stage 2: Risk Assessment with Gate Check"""
    print("\n=== STAGE 2: RISK ASSESSMENT ===")
    
    # Create structured input from Stage 1 output
    risk_input = f"""Business Risk Analysis Request:
    
Business: {document_review.business_name}
Owner: {document_review.applicant_name}
Loan Amount: ${document_review.loan_amount:,.2f}

Financial Data:
- Credit Score: {document_review.financial_summary.get('credit_score', 'Not provided')}
- Latest Annual Revenue: ${document_review.financial_summary.get('annual_revenue_latest', 0):,.2f}
- Monthly Expenses: ${document_review.financial_summary.get('monthly_expenses', 0):,.2f}
- Current Debt: ${document_review.financial_summary.get('current_debt', 0):,.2f}

Document Status: {document_review.document_status.value}
Missing Documents: {', '.join(document_review.missing_documents) if document_review.missing_documents else 'None'}

Please conduct comprehensive risk assessment."""
    
    response = get_completion(stage2_prompt, risk_input)
    print(f"Raw AI Response:\n{response}\n")
    
    try:
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            json_data = json.loads(json_match.group())
            
            risk_assessment = RiskAssessment(**json_data)
            print("✅ Stage 2 Gate Check: PASSED")
            return risk_assessment
        else:
            raise ValueError("No valid JSON found in response")
            
    except (json.JSONDecodeError, Exception) as e:
        print(f"❌ Stage 2 Gate Check: FAILED - {e}")
        return None

# Test Stage 2 (if Stage 1 succeeded)
if stage1_result:
    stage2_result = execute_stage2(stage1_result)
    if stage2_result:
        print(f"\n📊 Stage 2 Output Summary:")
        print(f"   Risk Level: {stage2_result.overall_risk_level.value}")
        print(f"   Risk Score: {stage2_result.risk_score}/100")
        print(f"   Credit Score: {stage2_result.credit_score}")
        print(f"   Debt Service Coverage: {stage2_result.debt_service_coverage:.2f}")


=== STAGE 2: RISK ASSESSMENT ===
Raw AI Response:
{
  "overall_risk_level": "medium",
  "credit_score": 685,
  "debt_service_coverage": 1.25,
  "risk_factors": ["Credit score in the good range", "Existing debt of $25,000", "Monthly expenses are significant relative to revenue"],
  "mitigating_factors": ["Established business with annual revenue of $195,000", "Debt service coverage is acceptable", "No missing documents"],
  "risk_score": 60
}

✅ Stage 2 Gate Check: PASSED

📊 Stage 2 Output Summary:
   Risk Level: medium
   Risk Score: 60/100
   Credit Score: 685
   Debt Service Coverage: 1.25


## Stage 3: Final Decision and Terms

Third stage synthesizes all previous analysis into final loan decision and terms.

In [9]:
# Stage 3: Loan Decision Specialist
stage3_prompt = """You are a senior loan officer making final lending decisions. Use the provided risk assessment to determine loan approval and terms.

DECISION CRITERIA:
- LOW RISK: Approve at prime rates, standard terms
- MEDIUM RISK: Conditional approval, higher rates, additional collateral/guarantees
- HIGH RISK: Generally decline, unless exceptional mitigating factors

RATE GUIDELINES:
- Excellent credit (740+): Prime + 1-2%
- Good credit (680-739): Prime + 2-4%  
- Fair credit (620-679): Prime + 4-6%
- Poor credit (<620): Prime + 6%+ or decline

Return ONLY a JSON object with this exact structure:
{
  "decision": "approve|conditional_approve|decline",
  "approved_amount": number_or_null,
  "interest_rate": number_or_null,
  "term_months": number_or_null,
  "conditions": ["list", "of", "conditions"],
  "reasoning": "detailed explanation"
}"""

def execute_stage3(risk_assessment: RiskAssessment):
    """Execute Stage 3: Final Decision with Gate Check"""
    print("\n=== STAGE 3: LOAN DECISION ===")
    
    decision_input = f"""Loan Decision Request:
    
Risk Assessment Results:
- Overall Risk Level: {risk_assessment.overall_risk_level.value}
- Risk Score: {risk_assessment.risk_score}/100
- Credit Score: {risk_assessment.credit_score}
- Debt Service Coverage Ratio: {risk_assessment.debt_service_coverage:.2f}

Risk Factors:
{chr(10).join('- ' + factor for factor in risk_assessment.risk_factors)}

Mitigating Factors:
{chr(10).join('- ' + factor for factor in risk_assessment.mitigating_factors)}

Please make final lending decision with appropriate terms and conditions."""
    
    response = get_completion(stage3_prompt, decision_input)
    print(f"Raw AI Response:\n{response}\n")
    
    try:
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            json_data = json.loads(json_match.group())
            
            loan_decision = LoanDecisionModel(**json_data)
            print("✅ Stage 3 Gate Check: PASSED")
            return loan_decision
        else:
            raise ValueError("No valid JSON found in response")
            
    except (json.JSONDecodeError, Exception) as e:
        print(f"❌ Stage 3 Gate Check: FAILED - {e}")
        return None

# Test Stage 3 (if Stage 2 succeeded)
if 'stage2_result' in locals() and stage2_result:
    stage3_result = execute_stage3(stage2_result)
    if stage3_result:
        print(f"\n📊 Stage 3 Output Summary:")
        print(f"   Decision: {stage3_result.decision.value}")
        if stage3_result.approved_amount:
            print(f"   Approved Amount: ${stage3_result.approved_amount:,.2f}")
            print(f"   Interest Rate: {stage3_result.interest_rate}%")
            print(f"   Term: {stage3_result.term_months} months")
        print(f"   Conditions: {len(stage3_result.conditions)} items")


=== STAGE 3: LOAN DECISION ===
Raw AI Response:
{
  "decision": "conditional_approve",
  "approved_amount": 50000,
  "interest_rate": 6.25,
  "term_months": 60,
  "conditions": [
    "Provide additional collateral equivalent to 10% of the loan amount",
    "Maintain a debt service coverage ratio above 1.2 during the loan term"
  ],
  "reasoning": "The loan is conditionally approved due to the medium risk level indicated by the credit score and existing debt. The business has an acceptable debt service coverage ratio and established revenue, which mitigates some risk. However, due to the existing debt and significant monthly expenses, additional collateral and monitoring of the debt service coverage ratio are required."
}

✅ Stage 3 Gate Check: PASSED

📊 Stage 3 Output Summary:
   Decision: conditional_approve
   Approved Amount: $50,000.00
   Interest Rate: 6.25%
   Term: 60 months
   Conditions: 2 items


## Complete Workflow Orchestration

Now let's put it all together in a single workflow function with error handling.

In [10]:
def execute_complete_workflow(application_text):
    """Execute the complete three-stage loan processing workflow"""
    print("🚀 STARTING COMPLETE LOAN PROCESSING WORKFLOW\n")
    
    workflow_results = {}
    
    # Stage 1: Document Review
    stage1_result = execute_stage1(application_text)
    if not stage1_result:
        print("❌ WORKFLOW FAILED: Stage 1 validation error")
        return None
    workflow_results['document_review'] = stage1_result
    
    # Stage 2: Risk Assessment  
    stage2_result = execute_stage2(stage1_result)
    if not stage2_result:
        print("❌ WORKFLOW FAILED: Stage 2 validation error") 
        return None
    workflow_results['risk_assessment'] = stage2_result
    
    # Stage 3: Final Decision
    stage3_result = execute_stage3(stage2_result)
    if not stage3_result:
        print("❌ WORKFLOW FAILED: Stage 3 validation error")
        return None
    workflow_results['final_decision'] = stage3_result
    
    print("✅ WORKFLOW COMPLETED SUCCESSFULLY!")
    return workflow_results

def display_workflow_summary(results):
    """Display a comprehensive summary of workflow results"""
    if not results:
        print("No results to display")
        return
        
    print("\n" + "="*60)
    print("📋 COMPLETE LOAN PROCESSING SUMMARY")
    print("="*60)
    
    doc_review = results['document_review']
    risk_assess = results['risk_assessment'] 
    decision = results['final_decision']
    
    print(f"\n🏢 BUSINESS INFORMATION:")
    print(f"   Business: {doc_review.business_name}")
    print(f"   Owner: {doc_review.applicant_name}")
    print(f"   Requested: ${doc_review.loan_amount:,.2f}")
    
    print(f"\n📄 DOCUMENT REVIEW:")
    print(f"   Status: {doc_review.document_status.value}")
    print(f"   Missing Items: {len(doc_review.missing_documents)}")
    
    print(f"\n⚠️ RISK ASSESSMENT:")
    print(f"   Risk Level: {risk_assess.overall_risk_level.value}")
    print(f"   Risk Score: {risk_assess.risk_score}/100")
    print(f"   Credit Score: {risk_assess.credit_score}")
    print(f"   DSC Ratio: {risk_assess.debt_service_coverage:.2f}")
    
    print(f"\n✅ FINAL DECISION:")
    print(f"   Decision: {decision.decision.value.upper()}")
    if decision.approved_amount:
        print(f"   Approved Amount: ${decision.approved_amount:,.2f}")
        print(f"   Interest Rate: {decision.interest_rate}%")
        print(f"   Term: {decision.term_months} months")
    print(f"   Conditions: {len(decision.conditions)} requirements")
    
    print(f"\n💡 REASONING:")
    print(f"   {decision.reasoning}")

# Execute complete workflow
print("🎯 RUNNING COMPLETE WORKFLOW DEMONSTRATION")
final_results = execute_complete_workflow(loan_application)
if final_results:
    display_workflow_summary(final_results)

🎯 RUNNING COMPLETE WORKFLOW DEMONSTRATION
🚀 STARTING COMPLETE LOAN PROCESSING WORKFLOW


=== STAGE 1: DOCUMENT REVIEW ===
Raw AI Response:
{
  "applicant_name": "Marina Santos",
  "business_name": "Marina's Coastal Café",
  "loan_amount": 85000,
  "document_status": "complete",
  "missing_documents": [],
  "financial_summary": {
    "annual_revenue_latest": 195000,
    "monthly_expenses": 12000,
    "current_debt": 25000,
    "credit_score": 685
  }
}

✅ Stage 1 Gate Check: PASSED

=== STAGE 2: RISK ASSESSMENT ===
Raw AI Response:
{
  "overall_risk_level": "medium",
  "credit_score": 685,
  "debt_service_coverage": 1.25,
  "risk_factors": ["Credit score in the good range", "Current debt level", "Monthly expenses are significant"],
  "mitigating_factors": ["Established business with annual revenue of $195,000", "Debt service coverage is acceptable", "No missing documents"],
  "risk_score": 55
}

✅ Stage 2 Gate Check: PASSED

=== STAGE 3: LOAN DECISION ===
Raw AI Response:
{
  "decision"

## Workflow vs. Single Prompt Comparison

Let's compare the quality and reliability of our chained approach versus the single prompt.

In [11]:
print("🔄 COMPARING APPROACHES: WORKFLOW vs. SINGLE PROMPT")
print("="*80)

print("\n📊 WORKFLOW APPROACH BENEFITS:")
print("✅ Structured, systematic analysis")
print("✅ Validated data flow between stages") 
print("✅ Specialized AI agents for each task")
print("✅ Error detection and handling at each gate")
print("✅ Consistent, parseable output format")
print("✅ Modular design - easy to modify individual stages")
print("✅ Audit trail of decision-making process")
print("✅ Professional-grade output ready for business use")

print("\n📊 SINGLE PROMPT LIMITATIONS:")
print("❌ Inconsistent output format")
print("❌ No intermediate validation")
print("❌ Difficult to debug failures")
print("❌ Jack-of-all-trades, master of none analysis")
print("❌ Hard to modify specific analysis components")
print("❌ Less reliable for production use")

print("\n🎯 WORKFLOW SUCCESS METRICS:")
if final_results:
    print(f"✅ All 3 stages completed successfully")
    print(f"✅ 3/3 Pydantic validation gates passed")
    print(f"✅ Structured JSON output at every stage")
    print(f"✅ Professional loan decision with complete reasoning")
    print(f"✅ Ready for integration with loan origination system")
else:
    print("❌ Workflow validation failed - would not occur with proper implementation")

print(f"\n💼 BUSINESS IMPACT:")
print(f"• Consistent loan processing quality")
print(f"• Reduced manual review time") 
print(f"• Improved audit compliance")
print(f"• Scalable automated underwriting")
print(f"• Lower operational risk")

🔄 COMPARING APPROACHES: WORKFLOW vs. SINGLE PROMPT

📊 WORKFLOW APPROACH BENEFITS:
✅ Structured, systematic analysis
✅ Validated data flow between stages
✅ Specialized AI agents for each task
✅ Error detection and handling at each gate
✅ Consistent, parseable output format
✅ Modular design - easy to modify individual stages
✅ Audit trail of decision-making process
✅ Professional-grade output ready for business use

📊 SINGLE PROMPT LIMITATIONS:
❌ Inconsistent output format
❌ No intermediate validation
❌ Difficult to debug failures
❌ Jack-of-all-trades, master of none analysis
❌ Hard to modify specific analysis components
❌ Less reliable for production use

🎯 WORKFLOW SUCCESS METRICS:
✅ All 3 stages completed successfully
✅ 3/3 Pydantic validation gates passed
✅ Structured JSON output at every stage
✅ Professional loan decision with complete reasoning
✅ Ready for integration with loan origination system

💼 BUSINESS IMPACT:
• Consistent loan processing quality
• Reduced manual review time


## Key Observations and Best Practices

### What We Learned:

1. **Specialization Wins**: Each AI agent focuses on one task, producing higher quality analysis than generalist approaches

2. **Validation Gates Are Critical**: Pydantic models ensure data quality and catch errors early in the process

3. **Structured Data Flow**: Well-defined interfaces between stages enable reliable workflow orchestration

4. **Error Handling**: Workflow design allows for graceful failure handling and debugging

5. **Production Ready**: Chained approach produces consistent, business-ready output

### Prompt Chaining Best Practices for Financial Services:

- **Clear Stage Separation**: Each stage should have a distinct, focused responsibility
- **Strong Validation**: Use Pydantic models to validate outputs before proceeding to next stage  
- **Structured Interfaces**: Define exact data formats for stage-to-stage communication
- **Error Recovery**: Plan for validation failures and retry mechanisms
- **Audit Trails**: Maintain complete records of each stage's analysis and decisions

### Real-World Applications:

- **Loan Origination Systems**: Automated underwriting with human oversight points
- **Insurance Claims Processing**: Multi-stage claim analysis and decision workflows
- **Investment Advisory**: Systematic client analysis, risk assessment, and recommendation generation
- **Compliance Monitoring**: Multi-step transaction analysis for regulatory reporting

### Next Steps:

In the upcoming exercise, you'll build your own multi-stage financial compliance workflow, applying these chaining principles to create systematic, validated analysis processes for suspicious activity reporting.